In [31]:
import pandas as pd
import numpy as np


### 분석데이터 불러오기

In [32]:
df = pd.read_csv('C:/Users/jaehong/Documents/GitHub/Project_0101/AS_Management_Data.csv')


### 분석데이터 확인하기

In [33]:
df

,접수날짜,제조일자,불량유형_대,불량유형_중,불량유형_소,원인,유상/무상,처리내역,수리부품,수리비용,수리비용청구여부,케이스,액정 Ass'y,컨넥터 Ass'y,MAIN Pcb Board,스위치 PCB Board Ass'y,SMPS Pcb Board,Fuse,Battery,검사자
0,2021-12-01,2021-10-17,외관,흠집,액정,검수미비,무상,제품교환,액정 Ass'y,80000,미청구,C커버,B패널,B통신,A테크,B텍,C테크놀러지,A상사,A리튬,김민지
1,2020-11-19,2020-07-11,외관,파손,케이스,사용자 과실,유상,부품교체,케이스,20000,20000,A케이스,B패널,B통신,C테크놀러지,A테크,B텍,A상사,C파워,이수현
2,2023-01-17,2020-11-17,외관,오염,케이스,사용자 과실,유상,부품교체,케이스,20000,20000,B사출,C코닝,A케이블,C테크놀러지,A테크,B텍,A상사,C파워,김민지
3,2021-01-22,2020-08-27,기능,화면,켜지지 않음,휴즈 끊어짐,무상,제품교환,Fuse,3000,미청구,B사출,C코닝,B통신,B텍,C테크놀러지,C테크놀러지,A상사,C파워,김민지
4,2023-08-11,2023-05-16,기능,화면,백화현상,액정 고장,무상,제품교환,액정 Ass'y,80000,미청구,A케이스,A액정,A케이블,B텍,B텍,C테크놀러지,A상사,C파워,김민지
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2024-06-01,2023-04-19,기능,화면,켜지지 않음,Pcb Board 고장,유상,부품교체,MAIN Pcb Board,50000,50000,A케이스,B패널,A케이블,A테크,A테크,B텍,A상사,A리튬,김민지
9996,2021-01-13,2020-11-14,외관,오염,케이스,사용자 과실,유상,부품교체,케이스,20000,20000,A케이스,C코닝,C텍,A테크,C테크놀러지,A테크,A상사,A리튬,이수현
9997,2022-12-19,2022-06-22,기능,화면,켜지지 않음,배터리 방전,무상,부품교체,Battery,30000,미청구,A케이스,C코닝,B통신,C테크놀러지,A테크,C테크놀러지,A상사,C파워,이수현
9998,2023-01-20,2022-09-24,외관,흠집,케이스,사용자 과실,유상,부품교체,케이스,20000,20000,A케이스,C코닝,B통신,B텍,A테크,B텍,A상사,C파워,이수현


### 분석데이터 head(3) 보기

In [34]:
df.head(3)

,접수날짜,제조일자,불량유형_대,불량유형_중,불량유형_소,원인,유상/무상,처리내역,수리부품,수리비용,수리비용청구여부,케이스,액정 Ass'y,컨넥터 Ass'y,MAIN Pcb Board,스위치 PCB Board Ass'y,SMPS Pcb Board,Fuse,Battery,검사자
0,2021-12-01,2021-10-17,외관,흠집,액정,검수미비,무상,제품교환,액정 Ass'y,80000,미청구,C커버,B패널,B통신,A테크,B텍,C테크놀러지,A상사,A리튬,김민지
1,2020-11-19,2020-07-11,외관,파손,케이스,사용자 과실,유상,부품교체,케이스,20000,20000,A케이스,B패널,B통신,C테크놀러지,A테크,B텍,A상사,C파워,이수현
2,2023-01-17,2020-11-17,외관,오염,케이스,사용자 과실,유상,부품교체,케이스,20000,20000,B사출,C코닝,A케이블,C테크놀러지,A테크,B텍,A상사,C파워,김민지


### 분석데이터 tail(3) 보기

In [41]:
df.tail(3)

,접수날,제조일,불량유,불량유,불량유,원인,유상/,처리내,수리부,수리비,수리비,케이스,액정,컨넥터,MAI,스위치,SMP,Fus,Bat,검사자
9997,2022-12-19,2022-06-22,기능,화면,켜지지 않음,배터리 방전,무상,부품교체,Battery,30000,미청구,A케이스,C코닝,B통신,C테크놀러지,A테크,C테크놀러지,A상사,C파워,이수현
9998,2023-01-20,2022-09-24,외관,흠집,케이스,사용자 과실,유상,부품교체,케이스,20000,20000,A케이스,C코닝,B통신,B텍,A테크,B텍,A상사,C파워,이수현
9999,2022-12-08,2022-11-26,외관,오염,액정,검수미비,무상,제품교환,액정 Ass'y,80000,미청구,C커버,B패널,A케이블,B텍,C테크놀러지,A테크,B파츠,C파워,이수현


### 결측치 보기

In [36]:
df.isna().sum()

접수날짜                   0
제조일자                   0
불량유형_대                 0
불량유형_중                 0
불량유형_소                 0
원인                     0
유상/무상                  0
처리내역                   0
수리부품                   0
수리비용                   0
수리비용청구여부               0
케이스                    0
액정 Ass'y               0
컨넥터 Ass'y              0
MAIN Pcb Board         0
스위치 PCB Board Ass'y    0
SMPS Pcb Board         0
Fuse                   0
Battery                0
검사자                    0
dtype: int64

In [ ]:
df